# ДЗ по "Сбор и разметка данных (семинары)"
## Семинар 9. Инструменты разметки наборов данных
* Домашнее задание:
- Задача 1.
Выберите датасет, который имеет отношение к вашей области интересов или исследований. Датасет должен содержать неструктурированные данные, требующие разметки для решения конкретной задачи, например, анализа настроений или распознавания именованных сущностей.

- Задача 2.
Выполните разметку на основе правил (rule-based labeling) на подмножестве выбранного датасета. Разработайте и реализуйте набор правил или условий, которые позволят автоматически присваивать метки данным на основе определенных шаблонов или критериев.

- Задача 3.
Выполните разметку вручную отдельного подмножества выбранного датасета с помощью выбранного вами инструмента разметки.

- Задача 4.
Объедините данные, размеченные вручную, с данными, размеченными на основе правил. Объедините два подмножества размеченных данных в один набор данных, сохранив при этом соответствующую структуру и целостность.

- Задача 5.
Обучите модель машинного обучения, используя объединенный набор размеченных данных. Разделите датасет на обучающий и тестовый наборы и используйте обучающий набор для обучения модели.

- Задача 6.
Оценить эффективность обученной модели на тестовом датасете. Используйте подходящие метрики оценки. Интерпретируйте результаты и проанализируйте эффективность модели в решении задачи разметки.



In [1]:
# Устанавливаем пакеты, если они не установлены
# !pip install pandas
# !pip install nltk
# !pip install textblob
# !pip install scikit-learn

In [2]:
import pandas as pd
import nltk
# Загружаем предобученную модель токенизации текста
nltk.download('punkt')
import csv
import os
import wget
from textblob import TextBlob
# from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\uc201\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## Загрузка датасетов

###  Загрузка датасета новостей lenta.ru

In [3]:
# Загрузим датасет с новостями с lenta.ru
news = pd.read_csv('news_lenta_ru.csv', engine='python', on_bad_lines='skip')

In [4]:
news

,url,title,text,topic,tags
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая
...,...,...,...,...,...
9995,https://lenta.ru/news/2000/07/10/kpp/,На границе Дагестана и Чечни будут стрелять в ...,Глава МВД Дагестана приказал нарядам милиции н...,Россия,Все
9996,https://lenta.ru/news/2000/07/10/gossovet/,Путин пересадит губернаторов в Госсовет,Представители Кремля ведут активный диалог с г...,Россия,Все
9997,https://lenta.ru/news/2000/07/10/clowns/,В Москву съедутся клоуны со всего мира,"В Москве 20 июля открывается фестиваль ""Планет...",Россия,Все
9998,https://lenta.ru/news/2000/07/10/women/,Старшая Уильямс выигрывает чемпионский титул,Чисто американским получился финал Уимблдонск...,Спорт,Все


### Загрузка тонального словаря

In [5]:
# Датасет тонального словаря
url = 'https://raw.githubusercontent.com/dkulagin/kartaslov/master/dataset/kartaslovsent/kartaslovsent.csv'
path = './kartaslovsent.csv'
if not os.path.isfile("kartaslovsent.csv"):
    # Загружаем данные
    wget.download(url, out=path)

In [6]:
df_tone_dictionary = pd.read_csv('kartaslovsent.csv', sep=';', on_bad_lines='skip')

In [7]:
df_tone_dictionary

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio
0,абажур,NEUT,0.08,0.185,0.037,0.580,0.198,0.00
1,аббатство,NEUT,0.10,0.192,0.038,0.578,0.192,0.00
2,аббревиатура,NEUT,0.08,0.196,0.000,0.630,0.174,0.00
3,абзац,NEUT,0.00,0.137,0.000,0.706,0.157,0.00
4,абиссинец,NEUT,0.28,0.151,0.113,0.245,0.491,0.19
...,...,...,...,...,...,...,...,...
46122,ёмкость,NEUT,0.00,0.167,0.000,0.690,0.143,0.00
46123,ёрзать,NGTV,-0.54,0.050,0.446,0.397,0.107,0.00
46124,ёрничать,NGTV,-0.79,0.078,0.529,0.236,0.157,0.00
46125,ёрш,NEUT,0.16,0.224,0.072,0.576,0.128,0.00


## Присваивание метки настроения заголовкам новостей

### Перемешаем и разделим датафрейм на три части (1-разметка в ручную, 2-разметка на основе правил, 3-разметка с помощью блиотеки TextBlob)

In [14]:
# Перемешаем и разделим датафрем на 3 части
title_news_1, title_news_2 = train_test_split(news['title'], train_size=5)
title_news_2, title_news_3 = train_test_split(title_news_2, train_size=50)

In [15]:
# Преобразуем полученые датасеты в датафреймы Pandas
df_title_news_1 = pd.DataFrame(title_news_1)
df_title_news_2 = pd.DataFrame(title_news_2)
df_title_news_3 = pd.DataFrame(title_news_3)

In [16]:
df_title_news_1.head()

,title
4767,Панк-рокера Элвиса Костелло продают в оцифрова...
3156,Наркоман Диего Марадона вышел из реанимации
1066,Треть российских компьютерных систем не готова...
4735,Корреспондент немецкой телекомпании Франк Хефл...
3958,Полиция Мексики взяла штурмом Национальный уни...


In [17]:
df_title_news_2.head()

,title
7405,В Чечне пропал самолет-разведчик СУ-24
6075,АРКО взяло в управление Башпромбанк и расплати...
7684,МПС готовится отдать пассажирские перевозки ча...
954,"Ставки в ""деле Аэрофлота"" выросли до 600.000.0..."
8059,"""Газпром"" сократит поставки газа энергетикам"


In [18]:
df_title_news_3.head()

,title
618,МВФ требует от Центробанка закрыть зарубежные ...
404,У Маркеса выявлен рак лимфы
7551,Жириновский поможет журналу Forbes обвинить Бе...
8065,Цены на бензин вырастут на 30 процентов?
5896,Черепков уверен в победе над владивостокским и...


### Опредиление функций

In [19]:
# Функция для присвоения метки настроения на основе наличия положительных или отрицательных слов
def get_sentiment_my_func(text, df_td=df_tone_dictionary):
    '''
    Функция вычисляет метрику настроения.

    :param text: текст который будем анализировать,
    :param df_td: тональный словарь,
    :return: возвращает оценку тональности: -0.35 (неготив) <= нейтрально >= (0.55) позитив 
    '''
    # Токенизация текста на отдельные слова
    words = nltk.word_tokenize(text.lower())
    count = 0      # Счетчик найденных слов
    sum_value = 0  # Сумма всех зачений весов найденных слов
    
    for word in words:
        if len(df_td[df_td.term == word]):
            count += 1
            df_filtered = df_td[df_td['term'] == word]
            value = df_filtered['value'].iloc[0]
            sum_value += value
    if count:
        sum_value = sum_value/count
  
    return sum_value   


In [20]:
# Функция присвоения метки настроения библиотеки TextBlob
def get_sentiment_blob(text):
    '''
    Функция вычисляет метрику настроения.

    :param text: текст который будем анализировать,
    :return: возвращает оценку тональности. 
    '''
    blob = TextBlob(text)
    return blob.sentiment.polarity

In [21]:
# Определение функции для присвоения метки настроения на основе оценки полярности настроения
def get_sentiment_label(score):
    '''
    Функция выводит метку настроения (негативный, позитивный или нейтральный).

    :param score: уровень метки настроения от -1 до 1,
    :return: возвращает оценку тональности. 
    '''
    if score < -0.35:
        return 'negative'  
    elif score > 0.55:
        return 'positive' 
    else:
        return 'neutral'

### Присваиваем метки

#### Присвоение меток настроения на основе правил

In [36]:
# Применение функции к столбцу 'title' датасета для получения полярности настроения каждого заголовка новости
df_title_news_2['sentiment'] = df_title_news_2['title'].apply(get_sentiment_my_func)
# Применение функции к столбцу 'sentiment' датасета для присвоения меток настроения
df_title_news_2['sentiment_label'] = df_title_news_2['sentiment'].apply(get_sentiment_label)

In [37]:
df_title_news_2.head()

,title,sentiment,sentiment_label
7405,В Чечне пропал самолет-разведчик СУ-24,0.00,neutral
6075,АРКО взяло в управление Башпромбанк и расплати...,0.34,neutral
7684,МПС готовится отдать пассажирские перевозки ча...,0.36,neutral
954,"Ставки в ""деле Аэрофлота"" выросли до 600.000.0...",0.00,neutral
8059,"""Газпром"" сократит поставки газа энергетикам",0.00,neutral


#### Присвоения меток настроения на основе модуля TextBlob

In [38]:
# Применение функции к столбцу 'title' датасета для получения полярности настроения каждого заголовка новости
df_title_news_3['sentiment'] = df_title_news_3['title'].apply(get_sentiment_blob)
# Применение функции к столбцу 'sentiment' датасета для присвоения меток настроения
df_title_news_3['sentiment_label'] = df_title_news_3['sentiment'].apply(get_sentiment_label)

In [39]:
df_title_news_3.head()

,title,sentiment,sentiment_label
618,МВФ требует от Центробанка закрыть зарубежные ...,0.0,neutral
404,У Маркеса выявлен рак лимфы,0.0,neutral
7551,Жириновский поможет журналу Forbes обвинить Бе...,0.0,neutral
8065,Цены на бензин вырастут на 30 процентов?,0.0,neutral
5896,Черепков уверен в победе над владивостокским и...,0.0,neutral


#### Далее

In [140]:
words = get_sentiment(reviews['text'].iloc[0], df_tone_dictionary)


In [141]:
words

('NEUT', -0.07416666666666667)

In [123]:
count = 0      # Счетчик найденных слов
sum_value = 0  # Сумма всех зачений весов найденных слов
for word in words:
    if len(loyln[loyln.term == word]):
        count += 1
        df_filtered = loyln[loyln['term'] == word]
        value = df_filtered['value'].iloc[0]
        sum_value += value
if count:
    sum_value = sum_value/count

('неприятель', -1.0)
('участие', 0.59)
('огонь', 0.27)
('попытка', 0.43)
('пробиться', 0.68)
('шоссе', 0.02)
('перемешивание', 0.0)
('часть', 0.0)
('неприятельский', -1.0)
('отряд', 0.0)
('преследовать', -0.43)
('инвалид', -0.45)
sum_value/count =-0.07416666666666667


In [64]:
r

,term,tag,value,pstv,ngtv,neut,dunno,pstvNgtvDisagreementRatio


In [90]:
# from corus import load_lenta2

# # Датасет с ноаостями lenta.ru
# url = 'https://github.com/natasha/corus/blob/master/data/lenta-ru-news.csv.bz2'
# path = './lenta-ru-news.csv.bz2'
# # if not os.path.isfile("lenta-ru-news.csv.bz2"):
#     # Загружаем данные
# # wget.download(url, out=path)
# # Создаем генратор с новостями с lenta.ru
# records = load_lenta2(path)
# # Добаляем текст новостей в словарь
# # news_dict = {'text': [next(records).text for _ in range(4_000)]}
# news_list = [{'url':item.url, 'title':item.title, 'text':item.text, 'topic':item.topic, 'tags':item.tags} for item in records]
# # next(records)

In [91]:
# len(news_list)

800975

In [178]:
# list2 = [news_list[i] for i in range(10_000)]

In [179]:
# len(list2)

10000

In [180]:
# # Создаём датафрем
# df = pd.DataFrame(list2)
# df.to_csv('news_lenta_ru.csv', encoding='utf-8', index=False)


In [181]:
# df

,url,title,text,topic,tags
0,https://lenta.ru/news/1914/09/16/hungarnn/,1914. Русские войска вступили в пределы Венгрии,Бои у Сопоцкина и Друскеник закончились отступ...,Библиотека,Первая мировая
1,https://lenta.ru/news/1914/09/16/lermontov/,1914. Празднование столетия М.Ю. Лермонтова от...,"Министерство народного просвещения, в виду про...",Библиотека,Первая мировая
2,https://lenta.ru/news/1914/09/17/nesteroff/,1914. Das ist Nesteroff!,"Штабс-капитан П. Н. Нестеров на днях, увидев в...",Библиотека,Первая мировая
3,https://lenta.ru/news/1914/09/17/bulldogn/,1914. Бульдог-гонец под Льежем,Фотограф-корреспондент Daily Mirror рассказыва...,Библиотека,Первая мировая
4,https://lenta.ru/news/1914/09/18/zver/,1914. Под Люблином пойман швабский зверь,"Лица, приехавшие в Варшаву из Люблина, передаю...",Библиотека,Первая мировая
...,...,...,...,...,...
9995,https://lenta.ru/news/2000/07/10/kpp/,На границе Дагестана и Чечни будут стрелять в ...,Глава МВД Дагестана приказал нарядам милиции н...,Россия,Все
9996,https://lenta.ru/news/2000/07/10/gossovet/,Путин пересадит губернаторов в Госсовет,Представители Кремля ведут активный диалог с г...,Россия,Все
9997,https://lenta.ru/news/2000/07/10/clowns/,В Москву съедутся клоуны со всего мира,"В Москве 20 июля открывается фестиваль ""Планет...",Россия,Все
9998,https://lenta.ru/news/2000/07/10/women/,Старшая Уильямс выигрывает чемпионский титул,Чисто американским получился финал Уимблдонск...,Спорт,Все


In [56]:
# len(news_dict['text'])

In [60]:
# for _ in range(2000):
#     next(records).text
#     # print(item.text)